In [2]:
import torch
import torch.nn as nn
from kasim_model import Kasim_Model
from kasim_tokenizer import Tokenizer   

k_tokenizer=Tokenizer("tokenizer.json")
sentence = "the capital of united states and the capital of france"
tokens = k_tokenizer.encode(sentence)
tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


k_model = Kasim_Model(vocab_size=len(k_tokenizer.vocab), embed_dim=4, context_length=32)
k_model


C:\Users\kasim\AppData\Local\Temp\ipykernel_7700\1443349770.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension


TypeError: Kasim_Model.__init__() missing 2 required positional arguments: 'num_heads' and 'num_layers'

NameError: name 'sentence_meanings_whith_attention' is not defined

In [3]:
import torch
from torch import nn
from kasim_causal_attention import KasimCausalAttention

class Kasim_Multi_Head_Attention(nn.Module):
    def __init__(self, embed_dim, num_heads,output_dim,context_length,dropout_rate=0):
        super().__init__()
        self.heads = nn.ModuleList([KasimCausalAttention(embed_dim,output_dim,context_length,dropout_rate) for _ in range(num_heads)])
            
    def forward(self, x):
        attention_outs=[]
        for head in self.heads:
            attention_outs.append(head(x))

        return torch.cat(attention_outs, dim=1)
        

In [4]:
mullti_head_attention = Kasim_Multi_Head_Attention(embed_dim=4, num_heads=2,output_dim=4,context_length=32)
out=mullti_head_attention(torch.randn(1,10,4))
out

tensor([[[ 9.0153e-01,  8.0140e-01,  6.3743e-01, -1.2156e-01],
         [ 5.9359e-01,  2.6023e-01,  7.0990e-01, -2.7597e-01],
         [ 5.9121e-01,  1.5560e-01,  8.0502e-01, -3.3606e-01],
         [ 4.7235e-01,  1.6914e-01,  7.4850e-01, -3.8456e-01],
         [ 4.3381e-01,  2.0413e-01,  7.0407e-01, -3.8365e-01],
         [ 4.2942e-01,  3.1018e-01,  6.2924e-01, -3.5284e-01],
         [ 3.4665e-01,  3.0350e-01,  5.5264e-01, -3.4712e-01],
         [ 2.4053e-01,  3.1924e-01,  4.1587e-01, -3.0777e-01],
         [ 2.3679e-01,  3.0131e-01,  4.2760e-01, -3.1444e-01],
         [ 6.6421e-02,  2.1244e-01,  2.8712e-01, -2.9416e-01],
         [ 5.5092e-01,  7.0129e-02, -1.2498e+00,  2.5312e-01],
         [ 4.4734e-01,  1.9993e-02, -6.2562e-01, -8.6194e-02],
         [ 4.7610e-01,  3.2873e-03, -5.5423e-01, -1.6547e-01],
         [ 4.7739e-01, -4.3303e-02, -5.7879e-01, -1.6255e-01],
         [ 4.7424e-01, -6.2760e-02, -6.0745e-01, -1.2920e-01],
         [ 4.7504e-01,  3.4339e-03, -7.9748e-01, -5.228

In [5]:
from kasim_layer_norm import KasimLayerNorm
norm_layer = KasimLayerNorm(4)
norm_layer(out)

tensor([[[ 0.8635,  0.6142,  0.2060, -1.6837],
         [ 0.7100, -0.1613,  1.0139, -1.5626],
         [ 0.6568, -0.3392,  1.1456, -1.4633],
         [ 0.5256, -0.1955,  1.1824, -1.5124],
         [ 0.4844, -0.0884,  1.1584, -1.5543],
         [ 0.4761,  0.1525,  1.0185, -1.6471],
         [ 0.3935,  0.2656,  1.0042, -1.6632],
         [ 0.2618,  0.5418,  0.8856, -1.6892],
         [ 0.2605,  0.4877,  0.9325, -1.6807],
         [-0.0069,  0.6461,  0.9800, -1.6193],
         [ 0.9358,  0.2381, -1.6775,  0.5036],
         [ 1.3302,  0.2122, -1.4769, -0.0656],
         [ 1.4502,  0.1714, -1.3365, -0.2850],
         [ 1.4712,  0.0890, -1.3326, -0.2276],
         [ 1.4488,  0.0483, -1.3722, -0.1249],
         [ 1.2467,  0.2113, -1.5470,  0.0890],
         [ 1.1350,  0.3038, -1.6095,  0.1707],
         [ 1.1830,  0.2213, -1.5869,  0.1825],
         [ 1.2354,  0.1350, -1.5560,  0.1855],
         [ 1.0811,  0.2149, -1.6342,  0.3383]]], grad_fn=<MulBackward0>)

In [6]:
import torch
import torch.nn as nn

# Diyelim ki elimizde 2 cümle (batch=2), her cümlede 4 token var (seq_len=4), embedding boyutu 3
x = torch.randn(2, 4, 3)  
print("Orijinal tensor:\n", x)

# LayerNorm tanımla (embedding boyutuna göre)
layer_norm = nn.LayerNorm(3)

# LayerNorm uygula
out = layer_norm(x)

print("\nLayerNorm sonrası tensor:\n", out)

# Ortalama ve varyans kontrolü
print("\nOrtalama:", out.mean(-1))   # Her token embeddinginin ortalaması ~0
print("Varyans:", out.var(-1))       # Her token embeddinginin varyansı ~1


Orijinal tensor:
 tensor([[[-0.0083,  1.3480, -0.1198],
         [ 0.7742,  0.0856,  0.8841],
         [-2.6429, -0.1946, -0.5494],
         [-0.4276, -1.1299,  0.8204]],

        [[ 0.4477,  2.0333, -0.6188],
         [-0.1941,  0.8728,  0.4899],
         [ 0.0187,  0.3004, -0.1455],
         [-0.3519,  0.7901, -0.5118]]])

LayerNorm sonrası tensor:
 tensor([[[-0.6219,  1.4109, -0.7890],
         [ 0.5458, -1.4027,  0.8569],
         [-1.4014,  0.8649,  0.5365],
         [-0.2255, -1.0963,  1.3218]],

        [[-0.1588,  1.2964, -1.1376],
         [-1.3225,  1.0951,  0.2274],
         [-0.2126,  1.3169, -1.1043],
         [-0.5647,  1.4052, -0.8405]]], grad_fn=<NativeLayerNormBackward0>)

Ortalama: tensor([[-3.9736e-08,  9.9341e-08,  1.9868e-08,  0.0000e+00],
        [-7.9473e-08, -1.4901e-08,  3.9736e-08,  3.9736e-08]],
       grad_fn=<MeanBackward1>)
Varyans: tensor([[1.5000, 1.4999, 1.5000, 1.5000],
        [1.5000, 1.4999, 1.4996, 1.5000]], grad_fn=<VarBackward0>)


In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

q_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
q_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

KeyboardInterrupt: 

In [8]:
q_model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [9]:
k_model

Kasim_Model(
  (embedding): Embedding(64, 4)
  (pos_embed): Embedding(32, 4)
  (self_attention): Kasim_Multi_Head_Attention(
    (multi_head_attention): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=4, out_features=4, bias=True)
    )
    (projection): Linear(in_features=4, out_features=4, bias=True)
  )
  (layer_norm): KasimRMSNorm()
  (mlp): KasimMLP(
    (gate_proj): Linear(in_features=4, out_features=4, bias=True)
    (up_proj): Linear(in_features=4, out_features=4, bias=True)
    (down_proj): Linear(in_features=4, out_features=4, bias=True)
    (gelu): KasimGelu()
  )
)

In [10]:
import torch
import torch.nn as nn
import math

class KasimGelu(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return 0.5 * x *(1 + torch.tanh(torch.sqrt(torch.tensor(2/ torch.pi)) * 
                                 (x + 0.044715 * torch.pow(x, 3))))
        

gelu = KasimGelu()
gelu(torch.tensor([-2.0, -1.0, 0.0, 1.0, 2.0]))        

tensor([-0.0454, -0.1588,  0.0000,  0.8412,  1.9546])

In [11]:
import torch
import torch.nn as nn

class KasimMLP(nn.Module):
    def __init__(self, embed_dim, hidden_dim):
        super().__init__()
        self.gate_proj = nn.Linear(embed_dim, hidden_dim)
        self.up_proj = nn.Linear(embed_dim,hidden_dim)
        self.down_proj = nn.Linear(hidden_dim, embed_dim)
        self.gelu = KasimGelu()
    def forward(self, x):
        gate=self.gate_proj(x)
        gate=self.gelu(gate)
        up=self.up_proj(x)
        fuse=gate*up
        outputs=self.down_proj(fuse)
        return outputs

In [1]:
import torch
import torch.nn as nn
from kasim_multi_head_attention import Kasim_Multi_Head_Attention
from kasim_RMSnorm import KasimRMSNorm
from kasim_MLP import KasimMLP

class KasimDecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, context_length):
        super().__init__()
        self.self_attention = Kasim_Multi_Head_Attention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            output_dim=embed_dim,  # Çıktı boyutu embed_dim olmalı
            context_length=context_length,
            dropout_rate=0.5
        )
        self.norm1 = KasimRMSNorm(embed_dim)
        self.mlp = KasimMLP(embed_dim, hidden_dim=4*embed_dim)  # hidden_dim genelde 4x
        self.norm2 = KasimRMSNorm(embed_dim)

    def forward(self, x):
        # Self-attention bloğu
        residual = x
        x = self.norm1(x)  # Pre-normalization
        x = self.self_attention(x)
        x = residual + x   # Residual connection

        # MLP bloğu
        residual = x
        x = self.norm2(x)  # Pre-normalization
        x = self.mlp(x)
        x = residual + x   # Residual connection

        return x

# Doğru boyutlarda test tensörü oluştur
batch_size = 2
seq_len = 4
embed_dim = 4

# (batch_size, seq_len, embed_dim) boyutunda tensor
example_tensor = torch.randn(batch_size, seq_len, embed_dim)

# Model parametrelerini düzelt
decoder_block = KasimDecoderBlock(
    embed_dim=embed_dim,
    num_heads=2,
    context_length=seq_len  # seq_len ile uyumlu
)

# Forward pass
output = decoder_block(example_tensor)

# Çıktı kontrolü
print(f"Input shape: {example_tensor.shape}")
print(f"Output shape: {output.shape}")
print(f"Output mean: {output.mean().item():.4f}")
print(f"Output std: {output.std().item():.4f}")

Input shape: torch.Size([2, 4, 4])
Output shape: torch.Size([2, 4, 4])
Output mean: 0.3486
Output std: 1.0652
